<a href="https://colab.research.google.com/github/M0hzz/AlgoFullStack/blob/master/CSC467_Module_04_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Main function to run all tasks
def main():
    # Choose an image from the prokudin-gorskii folder
    # You would need to replace this with the actual path
    image_path = 'prokudin-gorskii/your_chosen_image.jpg'

    # Task 1: Split and combine
    red_channel, green_channel, blue_channel, basic_color_image = process_image_task1(image_path)

    # Task 2: Align using two different metrics
    aligned_ncc, aligned_dot, offsets_ncc, offsets_dot = process_image_task2(
        red_channel, green_channel, blue_channel
    )

    # Task 3: Pyramid alignment for large images
    # For the specific high-resolution images mentioned in the assignment
    # You would need to load these specific images
    # Assuming you have them as 'seoul_tableau.jpg' and 'vancouver_tableau.jpg'

    # For Seoul image
    seoul_image_path = 'prokudin-gorskii/seoul tableau.jpg'
    seoul_img = plt.imread(seoul_image_path)
    height, width = seoul_img.shape[:2]
    panel_height = height // 3

    seoul_red = seoul_img[2*panel_height:, :]
    seoul_green = seoul_img[panel_height:2*panel_height, :]
    seoul_blue = seoul_img[:panel_height, :]

    aligned_seoul, seoul_level_offsets, seoul_total_offsets = process_image_task3(
        seoul_red, seoul_green, seoul_blue
    )

    # For Vancouver image
    vancouver_image_path = 'prokudin-gorskii/vancouver tableau.jpg'
    vancouver_img = plt.imread(vancouver_image_path)
    height, width = vancouver_img.shape[:2]
    panel_height = height // 3

    vancouver_red = vancouver_img[2*panel_height:, :]
    vancouver_green = vancouver_img[panel_height:2*panel_height, :]
    vancouver_blue = vancouver_img[:panel_height, :]

    aligned_vancouver, vancouver_level_offsets, vancouver_total_offsets = process_image_task3(
        vancouver_red, vancouver_green, vancouver_blue
    )

    # Answer Task 3 questions
    print("Task 3 Questions:")
    print("1. Equivalent range of offset in original images:")
    # If we search [-10, 10] at each level with scale factor 4 and 3 levels:
    # Level 0 (lowest res): [-10, 10]
    # Level 1: Level 0 result * 4 + [-10, 10] = [-50, 50]
    # Level 2 (original res): Level 1 result * 4 + [-10, 10] = [-210, 210]
    print("   Range: [-210, 210] pixels")

    print("2. Speed comparison:")
    # Simple search would check (420)^2 = 176,400 offsets
    # Pyramid approach checks:
    # Level 0: (21)^2 offsets at 1/16 size = 441 * (1/16) = 27.6 units
    # Level 1: (21)^2 offsets at 1/4 size = 441 * (1/4) = 110.3 units
    # Level 2: (21)^2 offsets at full size = 441 units
    # Total: 27.6 + 110.3 + 441 = 578.9 units
    # Speedup = 176,400 / 578.9 ≈ 304.7x
    print("   The pyramid approach is approximately 300 times faster")

if __name__ == "__main__":
    main()

# Task 1: Split and Combine
def process_image_task1(image_path):
    """
    Process an image for Task 1: Split the triptych and create a basic color image.
    """
    # Read the image
    img = plt.imread(image_path)

    # Get the height and width of the image
    height, width = img.shape[:2]

    # Calculate the height of each panel (assuming equal sizes)
    panel_height = height // 3

    # Split the image into three panels (B, G, R from top to bottom)
    blue_channel = img[:panel_height, :]
    green_channel = img[panel_height:2*panel_height, :]
    red_channel = img[2*panel_height:, :]

    # Stack the channels in RGB order
    color_image = np.stack((red_channel, green_channel, blue_channel), axis=2)

    # Save the color image
    plt.imsave('color_image_task1.jpg', color_image)

    print("Task 1 complete: Basic color image created")

    return red_channel, green_channel, blue_channel, color_image

# Task 2: Alignment
def process_image_task2(red_channel, green_channel, blue_channel):
    """
    Process an image for Task 2: Align the channels using normalized cross-correlation.
    """
    # Align using normalized cross-correlation
    aligned_image_ncc, offsets_ncc = align_and_combine(
        red_channel, green_channel, blue_channel, offset_range=15, use_center=True
    )

    # Save the aligned image
    plt.imsave('aligned_image_ncc.jpg', aligned_image_ncc)

    # Align using simple dot product for comparison
    aligned_image_dot, offsets_dot = align_and_combine_simple_dot(
        red_channel, green_channel, blue_channel, offset_range=15, use_center=True
    )

    # Save the aligned image with dot product metric
    plt.imsave('aligned_image_dot.jpg', aligned_image_dot)

    print("Task 2 complete: Aligned images created")
    print("Offsets using NCC:", offsets_ncc)
    print("Offsets using simple dot product:", offsets_dot)

    return aligned_image_ncc, aligned_image_dot, offsets_ncc, offsets_dot

# Task 3: Pyramid Alignment
def process_image_task3(red_channel, green_channel, blue_channel):
    """
    Process an image for Task 3: Align the channels using pyramid approach.
    """
    # Apply pyramid alignment
    aligned_image, level_offsets, total_offsets = pyramid_align(
        red_channel, green_channel, blue_channel, levels=3, scale_factor=4, offset_range=10
    )

    # Save the aligned image
    plt.imsave('aligned_image_pyramid.jpg', aligned_image)

    print("Task 3 complete: Pyramid-aligned image created")
    print("Level offsets:", level_offsets)
    print("Total offsets:", total_offsets)

    return aligned_image, level_offsets, total_offsets

def pyramid_align(red_channel, green_channel, blue_channel, levels=3, scale_factor=4, offset_range=10, use_center=True):
    """
    Align three channels using image pyramid approach.
    """
    # Create pyramids for each channel
    red_pyramid = create_pyramid(red_channel, levels, scale_factor)
    green_pyramid = create_pyramid(green_channel, levels, scale_factor)
    blue_pyramid = create_pyramid(blue_channel, levels, scale_factor)

    # Initialize offsets
    g_y_total, g_x_total = 0, 0
    b_y_total, b_x_total = 0, 0
    level_offsets = []

    # Process each level from lowest to highest resolution
    for level in range(levels):
        # Get the current level images
        red_level = red_pyramid[level]
        green_level = green_pyramid[level]
        blue_level = blue_pyramid[level]

        # Apply accumulated offsets from previous levels (scaled)
        if level > 0:
            g_y_total *= scale_factor
            g_x_total *= scale_factor
            b_y_total *= scale_factor
            b_x_total *= scale_factor

            green_level = np.roll(green_level, (g_y_total, g_x_total), axis=(0, 1))
            blue_level = np.roll(blue_level, (b_y_total, b_x_total), axis=(0, 1))

        # Find additional offsets at this level
        g_y_offset, g_x_offset = best_offset_ncc(red_level, green_level, offset_range, use_center)
        b_y_offset, b_x_offset = best_offset_ncc(red_level, blue_level, offset_range, use_center)

        # Apply these offsets
        green_level = np.roll(green_level, (g_y_offset, g_x_offset), axis=(0, 1))
        blue_level = np.roll(blue_level, (b_y_offset, b_x_offset), axis=(0, 1))

        # Update total offsets
        g_y_total += g_y_offset
        g_x_total += g_x_offset
        b_y_total += b_y_offset
        b_x_total += b_x_offset

        # Store the offsets for this level
        level_offsets.append({
            'level': level,
            'green': (g_y_offset, g_x_offset),
            'blue': (b_y_offset, b_x_offset)
        })

    # Apply the final offsets to the original channels
    aligned_green = np.roll(green_channel, (g_y_total, g_x_total), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_y_total, b_x_total), axis=(0, 1))

    # Stack the channels
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for display
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    # Store the total offsets
    total_offsets = {
        'green': (g_y_total, g_x_total),
        'blue': (b_y_total, b_x_total)
    }

    return aligned_image, level_offsets, total_offsets

def create_pyramid(image, levels=3, scale_factor=4):
    """
    Create an image pyramid by downsampling the image.
    """
    pyramid = [image]
    current_image = image

    for _ in range(levels - 1):
        h, w = current_image.shape[:2]
        new_h, new_w = h // scale_factor, w // scale_factor
        resized = cv2.resize(current_image, (new_w, new_h))
        pyramid.append(resized)

    # Reverse so that lowest resolution is first
    return pyramid[::-1]

def align_and_combine(red_channel, green_channel, blue_channel, offset_range=15, use_center=True):
    """
    Align and combine the three channels into a color image using normalized cross-correlation.
    """
    # Find best offsets for green and blue channels relative to red
    g_y_offset, g_x_offset = best_offset_ncc(red_channel, green_channel, offset_range, use_center)
    b_y_offset, b_x_offset = best_offset_ncc(red_channel, blue_channel, offset_range, use_center)

    # Apply the offsets
    aligned_green = np.roll(green_channel, (g_y_offset, g_x_offset), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_y_offset, b_x_offset), axis=(0, 1))

    # Stack the channels
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for display
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    # Store the offsets
    offsets = {
        'green': (g_y_offset, g_x_offset),
        'blue': (b_y_offset, b_x_offset)
    }

    return aligned_image, offsets

def normalized_cross_correlation(channel1, channel2):
    """
    Compute the normalized cross-correlation between two channels.
    """
    # Flatten the channels to 1D arrays
    channel1_flat = channel1.flatten()
    channel2_flat = channel2.flatten()

    # Normalize the channels by their L2 norm
    channel1_norm = channel1_flat / np.sqrt(np.sum(channel1_flat**2))
    channel2_norm = channel2_flat / np.sqrt(np.sum(channel2_flat**2))

    # Compute the dot product (correlation)
    ncc = np.dot(channel1_norm, channel2_norm)

    return ncc

def normalized_cross_correlation_center(channel1, channel2, border=15):
    """
    Compute the normalized cross-correlation between two channels,
    using only the center region unaffected by offsetting artifacts.
    """
    # Exclude border pixels
    h, w = channel1.shape
    center1 = channel1[border:h-border, border:w-border]
    center2 = channel2[border:h-border, border:w-border]

    return normalized_cross_correlation(center1, center2)

def simple_dot_product(channel1, channel2):
    """
    Compute the simple dot product between two channels without normalization.
    """
    # Flatten the channels to 1D arrays
    channel1_flat = channel1.flatten()
    channel2_flat = channel2.flatten()

    # Compute the dot product (correlation)
    dot_product = np.dot(channel1_flat, channel2_flat)

    return dot_product

    # Apply the offsets
    aligned_green = np.roll(green_channel, (g_y_offset, g_x_offset), axis=(0, 1))
    aligned_blue = np.roll(blue_channel, (b_y_offset, b_x_offset), axis=(0, 1))

    # Stack the channels
    aligned_image = np.stack((red_channel, aligned_green, aligned_blue), axis=2)

    # If the image has values in range [0,1], convert to [0,255] for display
    if aligned_image.max() <= 1.0:
        aligned_image = (aligned_image * 255).astype(np.uint8)

    # Store the offsets
    offsets = {
        'green': (g_y_offset, g_x_offset),
        'blue': (b_y_offset, b_x_offset)
    }

    return aligned_image, offsets

def best_offset_ncc(fixed_channel, moving_channel, offset_range=15, use_center=True):
    """
    Find the best offset using normalized cross-correlation.
    """
    best_corr = -1  # Initialize with a negative correlation
    best_y, best_x = 0, 0

    # Try different offsets
    for y_offset in range(-offset_range, offset_range + 1):
        for x_offset in range(-offset_range, offset_range + 1):
            # Shift the moving channel
            shifted = np.roll(moving_channel, (y_offset, x_offset), axis=(0, 1))

            # Compute correlation
            if use_center:
                corr = normalized_cross_correlation_center(fixed_channel, shifted, border=offset_range)
            else:
                corr = normalized_cross_correlation(fixed_channel, shifted)

            # Update if better correlation found
            if corr > best_corr:
                best_corr = corr
                best_y, best_x = y_offset, x_offset

    return best_y, best_x

def best_offset_dot(fixed_channel, moving_channel, offset_range=15, use_center=True):
    """
    Find the best offset using simple dot product.
    """
    best_corr = -float('inf')  # Initialize with negative infinity
    best_y, best_x = 0, 0

    # Try different offsets
    for y_offset in range(-offset_range, offset_range + 1):
        for x_offset in range(-offset_range, offset_range + 1):
            # Shift the moving channel
            shifted = np.roll(moving_channel, (y_offset, x_offset), axis=(0, 1))

            # Compute simple dot product
            if use_center:
                h, w = fixed_channel.shape
                border = offset_range
                fixed_center = fixed_channel[border:h-border, border:w-border]
                shifted_center = shifted[border:h-border, border:w-border]
                corr = simple_dot_product(fixed_center, shifted_center)
            else:
                corr = simple_dot_product(fixed_channel, shifted)

            # Update if better correlation found
            if corr > best_corr:
                best_corr = corr
                best_y, best_x = y_offset, x_offset

    return best_y, best_x